In [14]:
import urllib3
import mysql.connector as sql
from bs4 import BeautifulSoup
from datetime import date, timedelta

#Urllib3
http = urllib3.PoolManager()

def generate_date(prev_date):
    from datetime import date, timedelta
    date_list = []
    for i in range(1, prev_date):
        tanggal = date.today() - timedelta(i)
        date_list.append(tanggal)
    
    return date_list

def insert(title, image, url, date):
    db_connection = sql.connect(host='127.0.0.1', database='news_aggregator', user='root', password='')
    cursor = db_connection.cursor()
    
    #Try Except
    try:
        data_table = ("INSERT INTO data (TITLE, IMAGE, URL, DATE) VALUES (%s, %s, %s, %s)")
        data_value = (title, image, url, date)

        cursor.execute(data_table, data_value)
        db_connection.commit()

    except sql.IntegrityError:
        print('duplicate entry')
        
    cursor.close()
    db_connection.close()

In [23]:
def pull_data_hipwee(domain, pagination):
    
    for i in range(1, pagination):
        url = domain + str(i)
        print(url)

        #Get article from website
        req = http.request('GET', url)
        soup = BeautifulSoup(req.data, 'lxml')
        
        try:
            archive = soup.find('div', attrs={'class': 'archive-post'})
        except:
            print('archive not found')
            break
            
        try:
            containers = archive.find_all('div', attrs={'class':'row'})
        except:
            print('container not found')
            break

        for cont in range(0, len(containers)):
            container = soup.find_all('div', attrs={'class':'row'})[cont]

            try:
                boxes = container.find_all('div', attrs={'class': 'col-sm-4'})
            except:
                print('box not found')
                break

            for bo in range(0, len(boxes)):
                box = container.find_all('div', attrs={'class': 'col-sm-4'})[bo]

                #Get Title Article
                try:
                    title = box.find('h2').text
                except:
                    print('h2.text not found')
                    break

                #Get Image
                try:
                    image = box.find('img').get('srcset')
                except:
                    print('image not found')
                    break

                #Get URL Article
                try:
                    url = box.find('a').get('href')
                except:
                    print('href not found')
                    break

                #Get Date
                today = date.today()

                insert(title, image, url, today)

    return "done"

In [6]:
domain = 'http://www.hipwee.com/terbaru/?halaman='

In [22]:
pull_data_hipwee(domain, 5)

http://www.hipwee.com/terbaru/?halaman=1
Menyibak Keindahan Teluk Cinta di Pulau Labengki. Pesona Surga Tersembunyi yang Tak Bisa Kamu Dustai!
Lagi Ramai Soal Penipuan Calon Jemaah Umroh oleh First Travel, Eh Sudah Ada FTV-nya Aja. Epik!
6 Alasan Kenapa Banyak Bendera Nasional yang Mirip di Dunia. Bukan Karena Nggak Kreatif Lho
Tak Hanya Kisah Cinta, Tapi 6 Hal Ini pun Membuat Dirimu di Usia 20-an Kecewa dan Merugi
Beban Hidup Akibat High Heel Bisa Hilang dengan 12 Deretan Foto Sepatu Balerina yang Menawan Ini
Galak Is Not a Crime! 5 Hal Ini Bisa Jadi Alasan yang Bikin Kamu Dianggap Cewek Galak #HipweeJurnal
Aktif di Sosmed, Tapi Kesepian di Kehidupan Nyata. Banyak yang Nggak Sadar Lho, Ini 4 Tandanya!
Turis Malaysia Bikin Ulah di Jepang. Mereka Merusak Penginapan dan Buang Kotoran Sembarangan!
8 Pertanyaan yang Pasti Muncul dan Buatmu Gelisah, Saat Akan Menjalani LDR-an Pertama Kalinya
Belum Banyak yang Tahu, 4 Selebriti Ini Ternyata Punya Garis Keturunan Pahlawan Besar Indonesia!
Nya

'done'